# Viscosity Coefficients

## Prelude
In this notebook we will calculate the viscosity of the Yukawa OCP.

The YAML input file can be found at [input_file](https://raw.githubusercontent.com/murillo-group/sarkas/master/docs/examples/YOCP/input_files/yocp_transport.yaml) and this notebook at [notebook](https://raw.githubusercontent.com/murillo-group/sarkas/master/docs/examples/YOCP/YOCP_Transport_NB.ipynb).


In [ ]:
# Import the usual libraries
%pylab
%matplotlib inline

import os
plt.style.use('MSUstyle')

# Import sarkas
from sarkas.processes import Simulation, PostProcess, PreProcess


# Create the file path to the YAML input file
input_file_name = os.path.join('input_files', 'yocp_transport.yaml')

In [ ]:
# pre = PreProcess(input_file_name)
# pre.setup(read_yaml=True)
# pre.run(loops=150)

In [ ]:
# sim = Simulation(input_file_name)
# sim.setup(read_yaml=True)
# sim.run()

In [ ]:
postproc = PostProcess(input_file_name)
postproc.setup(read_yaml=True)
# postproc.parameters.verbose = True

In [ ]:
from sarkas.tools.observables import Thermodynamics, PressureTensor, HeatFlux, RadialDistributionFunction, VelocityAutoCorrelationFunction

In [ ]:
therm = Thermodynamics()
therm.setup(postproc.parameters, no_slices=3 )
therm.compute()
therm.grab_sim_data()
therm.temp_energy_plot(postproc)

## Pair Distribution Function

The first observable to calculate is always the RDF.

In [ ]:
rdf = RadialDistributionFunction()
rdf.setup(postproc.parameters, no_slices=3 )
rdf.parse()


In [ ]:
rdf.plot(scaling = rdf.a_ws, 
         y = ('C-C RDF', 'Mean'),
         xlabel = r'$r /a$')

## Pressure Tensor

The viscosity is obtained from the autocorrelation function of the Pressure Tensor $\overleftrightarrow{\mathcal P}$ whose elements are

\begin{equation}
\mathcal P_{\alpha\gamma}(t) = \frac{1}{V} \sum_{i}^{N} \left [ m_i v^{\alpha}_{i} v^{\gamma}_{i} -  \sum_{j > i} \frac{r_{ij}^{\alpha} r_{ij}^{\gamma} }{r_{ij}} \frac{d}{dr}\phi(r) \right ],
\end{equation}

where $r_{ij}^{\alpha}$ is the $\alpha$ component of the distance between particles $i$ and $j$. The first term is the kinetic term and the second term is the virial term, but it is often referred to as the potential contribution. The virial is calculated during the simulation phase and saved together with particles corrdinates. 

In order to check that our code are correct, let's verify some laws. 

The pressure of the system is calculated from $\mathcal P(t)= \frac1{3} {\rm Tr} \overleftrightarrow{\mathcal P}(t)$ and also from 

\begin{equation}
P = \frac{n}{\beta} - \frac{2\pi}{3} n^2 \int_0^{\infty} dr \, r^3 \frac{d\phi(r)}{dr} g(r)
\end{equation}

where $g(r)$ is the pair distribution function that we have already calculated.

Let's calculate the Pressure tensor and the pressure $\mathcal P$.

In [ ]:
pt = PressureTensor()
pt.setup(postproc.parameters, no_slices = 3)
# pt.compute()
pt.parse(acf_data=True)

As usual the data is saved in several dataframes. In this case we have 4 dataframes

* A dataframe for the values of each of the elements of the pressure tensor for each of the slices, `pt.dataframe_slices`
* A dataframe for the mean and std values of each of the elements of the pressure tensor, `pt.dataframe`
* A dataframe for the ACF of each pair $\langle \mathcal P_{\alpha\beta}(t)\mathcal P_{\mu\nu}(0) \rangle$ for each slice, `pt.dataframe_acf_slices`
* A dataframe for the mean and std of the ACF of each pair $\langle \mathcal P_{\alpha\beta}(t)\mathcal P_{\mu\nu}(0) \rangle$, `pt.dataframe_acf`

Let's look at `pt.dataframe` and at its columns

In [ ]:
pt.dataframe

Note that the Pressure $\mathcal P(t)$ is readily calculated and provided as a column of the dataframe.

Note also that there is a multitude of columns. This is because in dense plasmas it is useful to know the contribution of both the kinetic term and potential term separately, as such the columns of each dataframe contain the kinetic, the potential, and the total value of each $\mathcal P_{\alpha\beta}$ and their ACFs.

Let's plot the Pressure as a function of time

In [ ]:
# Let's plot it
t_wp = 2.0*np.pi/therm.total_plasma_frequency

p_id = pt.total_num_density / therm.beta_slice.mean()
ax = pt.plot( 
    scaling = (t_wp, p_id),
    y = ("Total","Pressure", "Mean"),
    xlabel = "Plasma cycles",
    ylabel = r"$ \beta P(t)/n$"
       )
ax.plot(pt.dataframe[("Species", "Quantity", 'Time')]/t_wp, pt.dataframe[("Total",'Pressure','Mean')].expanding().mean()/p_id )
ax.legend(['Pressure', 'Moving avg'])

In [ ]:
from sarkas.tools.observables import make_gaussian_plot


In [ ]:
pt.dataframe.columns

In [ ]:
make_gaussian_plot(pt.dataframe[("Species", "Quantity", 'Time')]/t_wp, pt.dataframe[("C",'Pressure Tensor XY','Mean')], "Pressure Tensor XY", "mks")

## Pressure from RDF

Let's now calculate the pressure from the integral of the RDF. This is obtained from the method `compute_from_rdf` of the `Thermodynamics` object. 

Looking at the documentation of this [method](:meth:`sarkas.tool.observables.Thermodynamics.compute_from_rdf`) we notice that it returns five values:
the Hartree and correlational terms between species :math:`A` and :math:`B` and the ideal pressure $n k_B T$. 

The total pressure is given from the sum of the three terms and should be equal to the 

$$ P = n k_BT + P_{\rm Hartree} + P_{\rm Corr} = {\operatorname {Mean} } \left \{ \mathcal P(t) \right \} $$

In [ ]:
nkT, _, _, p_h, p_c = therm.compute_from_rdf(rdf, postproc.potential)

P_rdf = nkT + p_h + p_c
P_trace = pt.dataframe[("Total","Pressure", "Mean")].mean()

print("The relative difference between the two methods is = {:.2f} %".format((P_rdf[0] - P_trace)*100/P_rdf[0] ) )

It seems that we have done a good job! 

### Sum rule

Let's now check that we have calculated the ACF correctly. The equal time ACFs of the elements of $\overleftrightarrow{\mathcal P}(t)$ obey the following sum rules

$$
\mathcal J_{zzzz}(0) = \frac 13 \sum_{\alpha}\left \langle \mathcal P_{\alpha\alpha}(0)\mathcal P_{\alpha\alpha}(0) \right \rangle  =  \frac{n}{\beta^2} \left [ 3 + \frac{2\beta}{15} I_1 + \frac \beta5 I_2 \right ] ,
$$ 
$$
\mathcal J_{zzxx}(0) = \frac 16 \sum_{\alpha} \sum_{\beta\neq\alpha} \left \langle \mathcal P_{\alpha\alpha}(0)\mathcal P_{\beta\beta}(0) \right \rangle = \frac{n}{\beta^2} \left [ 1 - \frac{2\beta}{5} I_1 + \frac \beta{15} I_2 \right ] ,
$$ 
$$
\mathcal J_{xyxy}(0) = \frac 16 \sum_{\alpha}\sum_{\beta \neq \alpha} \left \langle \mathcal P_{\alpha\beta}(0)\mathcal P_{\alpha\beta}(0) \right \rangle = \frac{n}{\beta^2} \left [ 1 + \frac{4\beta}{15} I_1 + \frac \beta{15} I_2 \right ] ,
$$ 

where

$$ 
I_1 = 2\pi n \int dr \, g(r) r^3 \frac{d\phi}{dr}, \quad I_2 = 2\pi n \int dr\, g(r) r^4 \frac{d^2\phi}{dr^2}.
$$

Notice that all three equal time ACF satisfy 

$$ \mathcal J_{zzzz}(0) - \mathcal J_{zzxx}(0) = 2 \mathcal J_{xyxy}(0) .$$

Let's look at the dataframe of the ACF first

In [ ]:
pt.dataframe_acf

Notice that in this case we have many more columns since now we have the ACF of the kinetic-kinetic, kinetic-potential, potential-kinetic, potential-potential, and the total ACF of each pair of elements.

Let's verify the sum rules.

In [ ]:
# Diagonal terms
column_zzzz = [
    ('Pressure Tensor ACF XXXX', 'Mean'),
     ('Pressure Tensor ACF YYYY', 'Mean'),
     ('Pressure Tensor ACF ZZZZ', 'Mean'),
]
J_zzzz_0 = pt.dataframe_acf[column_zzzz].iloc[0].mean()

# Cross-Diagonal Terms
column_zzxx = [
    ('Pressure Tensor ACF XXYY', 'Mean'),
    ('Pressure Tensor ACF XXZZ', 'Mean'),
    ('Pressure Tensor ACF YYZZ', 'Mean'),
]
J_zzxx_0 = pt.dataframe_acf[column_zzxx].iloc[0].mean()

# Cross Off Diagonal terms
column_xyxy = [
    ('Pressure Tensor ACF XYXY', 'Mean'),
    ('Pressure Tensor ACF XZXZ', 'Mean'),
    ('Pressure Tensor ACF YZYZ', 'Mean'),
]
J_xyxy_0 = pt.dataframe_acf[column_xyxy].iloc[0].mean()

# The units of J's are [Density *  Energy]^2

print('The isotropy condition : (J_zzzz_0 - J_zzxx_0 )/( 2*J_xyxy_0 ) = {:.4f}'.format( (J_zzzz_0 - J_zzxx_0)/(2.0 * J_xyxy_0)  ))

Not exactly 1 but pretty close.

Let's now verify the sum rules. These are calculated from the `pt.sum_rule` method

In [ ]:
h_r, c_r = rdf.compute_sum_rule_integrals(postproc.potential)
sigma_zzzz, sigma_zzxx, sigma_xyxy = pt.sum_rule(therm.beta_slice.mean(), rdf, postproc.potential)

In [ ]:
G_inf = J_xyxy_0*therm.beta_slice.mean()*rdf.box_volume
K_inf = 1.0/3.0*(J_zzzz_0 + 2.0* J_zzxx_0)*therm.beta_slice.mean()*rdf.box_volume

K_sr = (sigma_zzzz + 2.0*sigma_zzxx)/3.0

const = 1.0 #postproc.species[0].sigma**3/postproc.species[0].epsilon

print("G_inf = {:2.1f}, sum_rule = {:2.1f}, {:2.2f} %".format( G_inf* const, sigma_xyxy * const, 100*abs(G_inf -  sigma_xyxy) /G_inf) )
print("K_inf = {:2.1f}, sum_rule = {:2.1f}, {:2.2f} %".format( K_inf * const,  K_sr * const, 100*abs(K_inf -  K_sr) /K_inf) )

## Viscosity

The shear viscosity is calculated from the Green-Kubo relation

\begin{equation}
\eta = \frac{\beta V}{3} \sum_{\alpha} \sum_{\gamma \neq \alpha} \int_0^{\infty} dt \, \left \langle \mathcal P_{\alpha\gamma}(t) \mathcal P_{\alpha\gamma}(0) \right \rangle,
\end{equation}

where $\beta = 1/k_B T$, $\alpha,\gamma = {x, y, z}$.

The bulk viscosity is given by a similar relation

\begin{equation}
\eta_V = \beta V \int_0^{\infty}dt \,  \left \langle \delta \mathcal P(t) \delta \mathcal P(0) \right \rangle,
\end{equation}

where

\begin{equation}
\delta \mathcal P(t) = \mathcal P(t) - \left \langle \mathcal P  \right \rangle
\end{equation}

is the deviation of the scalar pressure.

In [ ]:
from sarkas.tools.transport import Viscosity

In [ ]:
tc = Viscosity()
tc.setup(postproc.parameters, observable=pt)
tc.compute(observable = pt)
# tc.parse(observable = pt, tc_name = "Viscosities")

In [ ]:
acf_str = "Delta Pressure ACF"
acf_avg = pt.dataframe_acf[("Pressure Bulk ACF", "Mean")]
acf_std = pt.dataframe_acf[("Pressure Bulk ACF", "Std")]

pq = "Bulk Viscosity"
tc_avg = tc.dataframe[(pq, "Mean")]
tc_std = tc.dataframe[(pq, "Std")]

In [ ]:
tc.dataframe

In [ ]:
fig, axes = tc.plot_tc(
    time = tc.dataframe[("Integration","Interval")].to_numpy(),
    acf_data=np.column_stack((acf_avg, acf_std)),
    tc_data=np.column_stack((tc_avg, tc_std)),
    acf_name=acf_str,
    tc_name="Bulk Viscosity",
    figname="{}_Plot.png".format("Bulk Viscosity"),
    show=False
)
axes[0].set(ylim = (-1, 1.05))
# axes[1].set(ylim = (-0.5, 1000 ) )

In [ ]:
def murillo_yvm(kappa, gamma):
    Ak = 0.46 *kappa**4/(1 + 0.44 * kappa**4)
    Bk = 1.01*np.exp(-0.92 * kappa)
    Ck = -3.7e-5 + 9.0e-4 * kappa - 2.9e-4*kappa**2
    
    gamma_ocp = Ak + Bk * gamma  + Ck*gamma**2
    lambda_yvm = 4.0 * np.pi/3.0 * (3.0 * gamma_ocp)**(3/2)
    I1 = 1.0/ (180 * gamma_ocp * np.pi **(3/2) )
    I2 = (0.49 - 2.23 * gamma_ocp**(-1/3) )/ (60 *np.pi**2)
    I3 = 0.241 * gamma_ocp**(1/9)/np.pi**(3/2)
    
    eta = lambda_yvm * I1 + (1 + lambda_yvm * I2)**2/(lambda_yvm * I3)
    return eta

In [ ]:
pq = "Shear Viscosity"
tc_avg = tc.dataframe[(pq, "Mean")]
tc_std = tc.dataframe[(pq, "Std")]


rescale = pt.total_plasma_frequency * pt.a_ws**2 * pt.species_masses[0] * pt.total_num_density
fig, ax = plt.subplots(1,1)
ax.plot(tc.dataframe[("Integration","Interval")].to_numpy()*1e12,
       tc_avg / rescale,
       label = r'$\mu$')

ax.fill_between(
    tc.dataframe[("Integration","Interval")].to_numpy()*1e12,
    (tc_avg - tc_std) / rescale,
    (tc_avg + tc_std) / rescale,
    alpha = 0.2)

ax.plot(tc.dataframe[("Integration","Interval")].to_numpy()*1e12,
       tc_avg.expanding().mean()/rescale,
       label = r'Moving avg')
ax.set(xlabel = r'Time lag $\tau$ [ps]',
      ylabel = r"Shear viscosity $\eta$",
      xscale=  'log'
      )
eta_yvm = murillo_yvm(postproc.potential.kappa, postproc.potential.coupling_constant)
ax.axhline(0.0654, ls = '--', c = 'r', label = "Daligault MD")
ax.axhline(eta_yvm, ls = ':', c = 'r', label = "Murillo YVM")

## Thermal Conductivity

In [ ]:
from sarkas.tools.observables import HeatFlux
from sarkas.tools.transport import ThermalConductivity

In [ ]:
ht = HeatFlux()
ht.setup(postproc.parameters, no_slices=3)
ht.compute(calculate_acf=True)

In [ ]:
thc = ThermalConductivity()
thc.setup(postproc.parameters, ht)
thc.compute(ht)